In [ ]:
import pandas as pd
import hvplot.pandas
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')

Load precipitation data

In [ ]:
pcp = pd.read_html('https://cdec.water.ca.gov/reportapp/javareports?name=DLYPCP')[0]
pcp.columns = pcp.columns.droplevel()
pcp = pcp[pcp.ID.notna()]
pcp = pcp[pcp.ID.str.match('^\\w{3}$')]
pcp['Month to Date'] = pd.to_numeric(pcp['Month to Date'], errors='coerce')
pcp['WY to Date'] = pd.to_numeric(pcp['WY to Date'], errors='coerce')

Load precipitation stations

In [ ]:
stations = pd.read_html('https://cdec.water.ca.gov/reportapp/javareports?name=RealPrecip')[0]
stations.columns = stations.columns.droplevel()
stations = stations[stations.ID.str.contains('^\\w{3}$')]
stations
stations.dtypes
stations = stations.astype({'LATITUDE':float,'LONGITUDE':float})
stations = stations[(stations.LATITUDE > 30)]# remove some junk lats

merge on station id

In [ ]:
pcpm = pcp.merge(stations, on='ID')
pcpm = pcpm.dropna(subset='WY to Date')

Map with color and size for precip to date

In [ ]:
def show_precip_map(precip_total_duration='WY to Date'):
    map = pcpm.hvplot.points('LONGITUDE','LATITUDE',
                                               geo=True, tiles='CartoLight',#tiles='ESRI',
                                               c=precip_total_duration, cmap='rainbow4', cnorm='eq_hist',
                                               hover_cols=['ID','Hydrologic Area  River Basin Station',
                                                           'Elev (FT)','Month to Date','WY to Date'],
                                               frame_width=500)#.opts(size=dim('WY to Date'))

    return map.opts(opts.Points(size=dim(precip_total_duration)*0.5, alpha=0.66, active_tools=['wheel_zoom']))#, alpha=0.1*dim('Month to Date')))

In [ ]:
#show_precip_map()

In [ ]:
import panel as pn
pn.extension()

In [ ]:
precip_total_duration = pn.widgets.Select(name='Precipitation Totals for', options=['Month to Date','WY to Date'])

view = pn.Column(precip_total_duration, pn.bind(show_precip_map, precip_total_duration))

view.servable(title='Precipitation Totals Map(CDEC)').show()